# AutoML Regression avec Azure ML service

<img src='https://cdn.thenewstack.io/media/2018/10/2e4f0988-az-ml-0.png'>

## Paramétrage

In [1]:
import sys
sys.version

'3.6.2 |Anaconda, Inc.| (default, Sep 30 2017, 18:42:57) \n[GCC 7.2.0]'

In [2]:
import datetime
now = datetime.datetime.now()
print(now)

2019-10-28 13:15:56.962136


In [3]:
# Version
import azureml.core
print("Version Azure ML service :", azureml.core.VERSION)

Version Azure ML service : 1.0.69


In [4]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
import azureml.dataprep as dprep
from azureml.train.automl import AutoMLConfig

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/runtime/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/runtime/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



## Expérimentation

In [5]:
ws = Workspace.from_config()

experiment_name = 'automl-regression'

project_folder = './sample_projects/automl-carprice-regression'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.69
Workspace Name,azuremlservice
Resource Group,azuremlserviceresourcegroup
Location,westeurope
Project Directory,./sample_projects/automl-carprice-regression
Experiment Name,automl-regression


## Définition AML Compute


In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "automlcl"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 6)

    # Create the cluster.
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min_node_count is provided, it will use the scale settings for the cluster.
    compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
     # For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.


## Data
This uses scikit-learn's [carprice.csv](https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/carprice.csv) method.

In [7]:
# You can use `auto_read_file` which intelligently figures out delimiters and datatypes of a file.
# The data referenced here is copied into a local temporary directory.
# You can also use `read_csv` and `to_*` transformations to read (with overridable delimiter)
# and convert column types manually.
example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/carprice.csv'
dflow = dprep.auto_read_file(example_data).skip(1)  # Remove the header row.
dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
symboling,FieldType.INTEGER,-2.00,3.00,204.00,0.00,204.00,0.00,0.00,0.00,6,-2.00,-1.00,-1.00,0.00,1.00,2.00,3.00,3.00,3.00,0.82,1.24,1.54,0.21,-0.69,3.00,-2.00
normalized-losses,FieldType.STRING,101,?,204.00,0.00,204.00,0.00,0.00,0.00,52,,,,,,,,,,,,,,,,
make,FieldType.STRING,alfa-romero,volvo,204.00,0.00,204.00,0.00,0.00,0.00,22,,,,,,,,,,,,,,,,
fuel-type,FieldType.STRING,diesel,gas,204.00,0.00,204.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,,,
aspiration,FieldType.STRING,std,turbo,204.00,0.00,204.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,,,
num-of-doors,FieldType.STRING,?,two,204.00,0.00,204.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
body-style,FieldType.STRING,convertible,wagon,204.00,0.00,204.00,0.00,0.00,0.00,5,,,,,,,,,,,,,,,,
drive-wheels,FieldType.STRING,4wd,rwd,204.00,0.00,204.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
engine-location,FieldType.STRING,front,rear,204.00,0.00,204.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,,,
wheel-base,FieldType.DECIMAL,86.60,120.90,204.00,0.00,204.00,0.00,0.00,0.00,53,86.60,93.70,93.70,94.50,96.98,102.40,110.60,115.60,120.90,98.81,5.99,35.93,1.05,0.94,114.20,86.60


In [8]:
dflow = dflow.to_number(['price', 'bore', 'stroke', 'horsepower', 'peak-rpm'])
dflow = dflow.drop_errors('price')
dflow.head(5)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111.00,5000.00,21,27,16500.00
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,...,152,mpfi,2.68,3.47,9.00,154.00,5000.00,19,26,16500.00
2,2,164,audi,gas,std,four,sedan,fwd,front,99.80,...,109,mpfi,3.19,3.40,10.00,102.00,5500.00,24,30,13950.00
3,2,164,audi,gas,std,four,sedan,4wd,front,99.40,...,136,mpfi,3.19,3.40,8.00,115.00,5500.00,18,22,17450.00
4,2,?,audi,gas,std,two,sedan,fwd,front,99.80,...,136,mpfi,3.19,3.40,8.50,110.00,5500.00,19,25,15250.00


In [9]:
dflow.dtypes

symboling                FieldType.INTEGER
normalized-losses        FieldType.STRING
make                     FieldType.STRING
fuel-type                FieldType.STRING
aspiration               FieldType.STRING
num-of-doors             FieldType.STRING
body-style               FieldType.STRING
drive-wheels             FieldType.STRING
engine-location          FieldType.STRING
wheel-base               FieldType.DECIMAL
length                   FieldType.DECIMAL
width                    FieldType.DECIMAL
height                   FieldType.DECIMAL
curb-weight              FieldType.INTEGER
engine-type              FieldType.STRING
num-of-cylinders         FieldType.STRING
engine-size              FieldType.INTEGER
fuel-system              FieldType.STRING
bore                     FieldType.DECIMAL
stroke                   FieldType.DECIMAL
compression-ratio        FieldType.DECIMAL
horsepower               FieldType.DECIMAL
peak-rpm                 FieldType.DECIMAL
city-mpg              

In [10]:
dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
symboling,FieldType.INTEGER,-2.00,3.00,200.00,0.00,200.00,0.00,0.00,0.00,6,-2.00,-1.00,-1.00,0.00,1.00,2.00,3.00,3.00,3.00,0.83,1.25,1.56,0.20,-0.72,3.00,-2.00
normalized-losses,FieldType.STRING,101,?,200.00,0.00,200.00,0.00,0.00,0.00,52,,,,,,,,,,,,,,,,
make,FieldType.STRING,alfa-romero,volvo,200.00,0.00,200.00,0.00,0.00,0.00,22,,,,,,,,,,,,,,,,
fuel-type,FieldType.STRING,diesel,gas,200.00,0.00,200.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,,,
aspiration,FieldType.STRING,std,turbo,200.00,0.00,200.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,,,
num-of-doors,FieldType.STRING,?,two,200.00,0.00,200.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
body-style,FieldType.STRING,convertible,wagon,200.00,0.00,200.00,0.00,0.00,0.00,5,,,,,,,,,,,,,,,,
drive-wheels,FieldType.STRING,4wd,rwd,200.00,0.00,200.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
engine-location,FieldType.STRING,front,rear,200.00,0.00,200.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,,,
wheel-base,FieldType.DECIMAL,86.60,120.90,200.00,0.00,200.00,0.00,0.00,0.00,52,86.60,93.70,93.70,94.50,96.98,102.40,111.00,115.60,120.90,98.85,6.04,36.46,1.03,0.87,114.20,86.60


In [11]:
#data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/carprice.csv"
#dflow = dprep.auto_read_file(data)
dflow.get_profile()
X = dflow.drop_columns(columns=['symboling','normalized-losses','price'])
y = dflow.keep_columns(columns=['price'], validate_column_exists=True)
X_train, X_test = X.random_split(percentage=0.8, seed=223)
y_train, y_test = y.random_split(percentage=0.8, seed=223) 
dflow.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111.00,5000.00,21,27,16500.00
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,...,152,mpfi,2.68,3.47,9.00,154.00,5000.00,19,26,16500.00
2,2,164,audi,gas,std,four,sedan,fwd,front,99.80,...,109,mpfi,3.19,3.40,10.00,102.00,5500.00,24,30,13950.00
3,2,164,audi,gas,std,four,sedan,4wd,front,99.40,...,136,mpfi,3.19,3.40,8.00,115.00,5500.00,18,22,17450.00
4,2,?,audi,gas,std,two,sedan,fwd,front,99.80,...,136,mpfi,3.19,3.40,8.50,110.00,5500.00,19,25,15250.00
5,1,158,audi,gas,std,four,sedan,fwd,front,105.80,...,136,mpfi,3.19,3.40,8.50,110.00,5500.00,19,25,17710.00
6,1,?,audi,gas,std,four,wagon,fwd,front,105.80,...,136,mpfi,3.19,3.40,8.50,110.00,5500.00,19,25,18920.00
7,1,158,audi,gas,turbo,four,sedan,fwd,front,105.80,...,131,mpfi,3.13,3.40,8.30,140.00,5500.00,17,20,23875.00
8,2,192,bmw,gas,std,two,sedan,rwd,front,101.20,...,108,mpfi,3.50,2.80,8.80,101.00,5800.00,23,29,16430.00
9,0,192,bmw,gas,std,four,sedan,rwd,front,101.20,...,108,mpfi,3.50,2.80,8.80,101.00,5800.00,23,29,16925.00


In [12]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE


cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]'], conda_packages=['numpy'])
conda_run_config.environment.python.conda_dependencies = cd

## AutoML Regression

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], targets values.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|

**_You can find more information about primary metrics_** [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)
                                                            

In [13]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 5,
    "n_cross_validations": 3,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "max_concurrent_iterations": 3,
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl.log',
                             path = project_folder,
                             X = X_train,
                             y = y_train,
                             **automl_settings
                            )


In [14]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_b9b2f955-8850-4505-8c0e-f919d6b79ed8
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       FIXED
DESCRIPTION:  The training data had the following missing values which were resolved.

Please review your data source for data quality issues and possibly filter out the rows with these missing values.

If the missing values are expected, you can either accept the above imputation, or implement

In [16]:
local_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-regression,AutoML_b9b2f955-8850-4505-8c0e-f919d6b79ed8,automl,Completed,Link to Azure Portal,Link to Documentation


## Résultats AutoML

In [17]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

A Jupyter Widget

In [18]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4
explained_variance,0.77,0.86,0.83,0.85,0.84
mean_absolute_error,2332.37,1833.98,2128.09,1910.54,1933.73
mean_absolute_percentage_error,16.49,13.03,15.99,13.62,13.52
median_absolute_error,1204.38,983.85,1289.29,1145.83,1063.19
normalized_mean_absolute_error,0.06,0.05,0.05,0.05,0.05
normalized_median_absolute_error,0.03,0.02,0.03,0.03,0.03
normalized_root_mean_squared_error,0.09,0.07,0.08,0.07,0.08
normalized_root_mean_squared_log_error,0.09,0.08,0.09,0.08,0.08
r2_score,0.77,0.85,0.83,0.85,0.83
root_mean_squared_error,3575.35,2818.76,3099.62,2883.63,3023.52


## Best Model

In [19]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automl-regression,
Id: AutoML_b9b2f955-8850-4505-8c0e-f919d6b79ed8_3,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...          flatten_transform=None,
               weights=[0.6666666666666666, 0.3333333333333333]))]),
          stddev=None)


### Changement de métrique

In [20]:
lookup_metric = "r2_score"
best_run, fitted_model = local_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

Run(Experiment: automl-regression,
Id: AutoML_b9b2f955-8850-4505-8c0e-f919d6b79ed8_1,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('MaxA...timators=25, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
          stddev=None)


### Choix itération particulière

In [21]:
iteration = 3
iter_run, iter_model = local_run.get_output(iteration = iteration)
print(iter_run)
print(iter_model)

Run(Experiment: automl-regression,
Id: AutoML_b9b2f955-8850-4505-8c0e-f919d6b79ed8_3,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...          flatten_transform=None,
               weights=[0.6666666666666666, 0.3333333333333333]))]),
          stddev=None)


> Fin